In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

# 1. Load dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 2. Preprocess data: reshape to include channel and normalize to [0,1]
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test  = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# 3. Set up data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,      # small rotations up to 15 degrees
    width_shift_range=0.1,  # translate horizontally by ±10% of width
    height_shift_range=0.1, # translate vertically by ±10% of height
    zoom_range=0.1,         # zoom in/out by ±10%
    horizontal_flip=True    # random horizontal flip
    # Note: vertical_flip is avoided to keep orientation realistic
)
datagen.fit(X_train)  # fit generator (not always necessary here, but required if using featurewise stats)

# 4. Build the CNN model (example architecture)
model = Sequential([
    Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, kernel_size=(3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # dropout for regularization
    Dense(10, activation='softmax')
])

# 5. Compile the model with an improved optimizer and appropriate loss
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
# (Alternatively, use Nadam for even potentially higher accuracy: tf.keras.optimizers.Nadam(lr=0.001))
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 6. Define learning rate scheduler and early stopping callbacks
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
early_stop   = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# 7. Train the model with data augmentation
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=64),
    steps_per_epoch = len(X_train) // 64,              # number of batches per epoch
    validation_data = (X_test, y_test),
    epochs=30,                                         # set upper bound; training may stop earlier due to early stopping
    callbacks=[lr_scheduler, early_stop]
)

# After training, evaluate on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/30


c:\Users\Urban\.conda\envs\kodilla-edu\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Urban\.conda\envs\kodilla-edu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


937/937 ━━━━━━━━━━━━━━━━━━━━ 23s 23ms/step - accuracy: 0.5895 - loss: 1.1214 - val_accuracy: 0.7792 - val_loss: 0.5573 - learning_rate: 0.0010
Epoch 2/30
  1/937 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.7344 - loss: 0.6709

c:\Users\Urban\.conda\envs\kodilla-edu\lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.7344 - loss: 0.6709 - val_accuracy: 0.7799 - val_loss: 0.5571 - learning_rate: 0.0010
Epoch 3/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step - accuracy: 0.7496 - loss: 0.6695 - val_accuracy: 0.8260 - val_loss: 0.4694 - learning_rate: 0.0010
Epoch 4/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 931us/step - accuracy: 0.8125 - loss: 0.6107 - val_accuracy: 0.8288 - val_loss: 0.4644 - learning_rate: 0.0010
Epoch 5/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 25s 27ms/step - accuracy: 0.7760 - loss: 0.6008 - val_accuracy: 0.8340 - val_loss: 0.4378 - learning_rate: 0.0010
Epoch 6/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 1s 921us/step - accuracy: 0.9062 - loss: 0.4174 - val_accuracy: 0.8342 - val_loss: 0.4351 - learning_rate: 0.0010
Epoch 7/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.7975 - loss: 0.5482 - val_accuracy: 0.8493 - val_loss: 0.4021 - learning_rate: 0.0010
Epoch 8/30
937/937 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - accuracy: 0.8281 - loss: 0.3767